In [13]:
# importando modulos necesarios
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import sympy
from scipy import integrate

# imprimir con notación matemática.
sympy.init_printing(use_latex='mathjax')

In [17]:
# Ejemplo de transformada de Laplace
# Defino las incognitas
t = sympy.symbols("t", positive=True)
y = sympy.Function("y")

# Defino la ecuación
edo =1*y(t).diff(t,t) + 20*y(t).diff(t) + (200)*y(t)-150*sympy.sin(10*t)
sympy.Eq(edo, 0)

                                          2           
                              d          d            
200⋅y(t) - 150⋅sin(10⋅t) + 20⋅──(y(t)) + ───(y(t)) = 0
                              dt           2          
                                         dt           

In [15]:
# simbolos adicionales.
s, Y = sympy.symbols("s, Y", real=True)

In [32]:
# Calculo la transformada de Laplace
L_edo = sympy.laplace_transform(edo, t, s, noconds=True)
sympy.Eq(L_edo)

TypeError: Equality.__new__() missing 1 required positional argument: 'rhs'

In [4]:
def laplace_transform_derivatives(e):
    """
    Evalua las transformadas de Laplace de derivadas de funciones sin evaluar.
    """
    if isinstance(e, sympy.LaplaceTransform):
        if isinstance(e.args[0], sympy.Derivative):
            d, t, s = e.args
            n = len(d.args) - 1
            return ((s**n) * sympy.LaplaceTransform(d.args[0], t, s) -
                    sum([s**(n-i) * sympy.diff(d.args[0], t, i-1).subs(t, 0)
                         for i in range(1, n+1)]))

    if isinstance(e, (sympy.Add, sympy.Mul)):
        t = type(e)
        return t(*[laplace_transform_derivatives(arg) for arg in e.args])

    return e

In [21]:
# Aplicamos la nueva funcion para evaluar las transformadas de Laplace
# de derivadas
L_edo_2 = laplace_transform_derivatives(L_edo)
sympy.Eq(L_edo_2, 0)

 2                                                                             ↪
s ⋅LaplaceTransform(y(t), t, s) + 20⋅s⋅LaplaceTransform(y(t), t, s) - s⋅y(0) + ↪
                                                                               ↪
                                                                               ↪

↪                                               ⎛d       ⎞│        1500      
↪  200⋅LaplaceTransform(y(t), t, s) - 20⋅y(0) - ⎜──(y(t))⎟│    - ──────── = 0
↪                                               ⎝dt      ⎠│t=0    2          
↪                                                                s  + 100    

In [22]:
# reemplazamos la transfomada de Laplace de y(t) por la incognita Y
# para facilitar la lectura de la ecuación.
L_edo_3 = L_edo_2.subs(sympy.laplace_transform(y(t), t, s), Y)
sympy.Eq(L_edo_3, 0)

 2                                                                             ↪
s ⋅LaplaceTransform(y(t), t, s) + 20⋅s⋅LaplaceTransform(y(t), t, s) - s⋅y(0) + ↪
                                                                               ↪
                                                                               ↪

↪                                               ⎛d       ⎞│        1500      
↪  200⋅LaplaceTransform(y(t), t, s) - 20⋅y(0) - ⎜──(y(t))⎟│    - ──────── = 0
↪                                               ⎝dt      ⎠│t=0    2          
↪                                                                s  + 100    

In [8]:
# Definimos las condiciones iniciales
ics = {y(0): 0, y(t).diff(t).subs(t, 0): -3}
ics

⎧         ⎛d       ⎞│       ⎫
⎨y(0): 0, ⎜──(y(t))⎟│   : -3⎬
⎩         ⎝dt      ⎠│t=0    ⎭

In [23]:
# Aplicamos las condiciones iniciales
L_edo_4 = L_edo_3.subs(ics)
sympy.Eq(L_edo_4, 0)

 2                                                                             ↪
s ⋅LaplaceTransform(y(t), t, s) + 20⋅s⋅LaplaceTransform(y(t), t, s) + 200⋅Lapl ↪
                                                                               ↪
                                                                               ↪

↪                                  1500      
↪ aceTransform(y(t), t, s) + 3 - ──────── = 0
↪                                 2          
↪                                s  + 100    

In [28]:
# Resolvemos la ecuación y arribamos a la Transformada de Laplace
# que es equivalente a nuestra ecuación diferencial
print("Equation to solve:")
display(sympy.Eq(L_edo_4, 0))
Y_sol = sympy.solve(L_edo_4, Y)
print("Solution for Y_sol:")
display(Y_sol)
if Y_sol:
    sympy.Eq(Y, Y_sol[0])
else:
    print("No solution found for Y.")

Equation to solve:


 2                                                                             ↪
s ⋅LaplaceTransform(y(t), t, s) + 20⋅s⋅LaplaceTransform(y(t), t, s) + 200⋅Lapl ↪
                                                                               ↪
                                                                               ↪

↪                                  1500      
↪ aceTransform(y(t), t, s) + 3 - ──────── = 0
↪                                 2          
↪                                s  + 100    

Solution for Y_sol:


[]

No solution found for Y.


In [29]:
# Por último, calculamos al inversa de la Transformada de Laplace que
# obtuvimos arriba, para obtener la solución de nuestra ecuación diferencial.
if Y_sol:
    y_sol = sympy.inverse_laplace_transform(Y_sol[0], s, t)
    print("Solution for y(t):")
    sympy.Eq(y(t), y_sol)
else:
    print("Cannot compute inverse Laplace transform without a solution for Y.")
    y_sol = None # Set y_sol to None to avoid NameError in the next cell

Cannot compute inverse Laplace transform without a solution for Y.


In [30]:
# Comprobamos la solución.
if y_sol is not None:
    print("Checking the solution with initial conditions:")
    sympy.Eq((y_sol.subs(t, 0), sympy.diff(y_sol).subs(t, 0)), (ics[y(0)], ics[y(t).diff(t).subs(t, 0)]))
else:
    print("Cannot check solution as y_sol was not computed.")

Cannot check solution as y_sol was not computed.


Actividad 1

In [45]:
# Importar la biblioteca necesaria
import sympy as sp
from sympy import latex

# Definir la variable
s = sp.symbols('s')

# Definir K como una constante
K = sp.symbols('K')

# Resultado de L{K}
L_K = K / s

# Create a function symbol for Laplace transform for display
L_sym = sp.Function('L')

# Imprimir la ecuación y el resultado en formato LaTeX
display(sp.Eq(L_sym(K), L_K))

       K
L(K) = ─
       s

Actividad 2

In [48]:
# Definir la variable t
t = sp.symbols('t')
s = sp.symbols('s') # Ensure s is defined here as well

# Resultado de L{t}
S_t = sp.laplace_transform(t, t, s)[0]  # Esto es 1/s^2

# Create a function symbol for Laplace transform for display
L_sym = sp.Function('L')

# Imprimir la ecuación y el resultado en formato LaTeX
display(sp.Eq(L_sym(t), S_t))

       1 
L(t) = ──
        2
       s 

Actividad 3

In [52]:
# Importar la biblioteca necesaria
import sympy as sp
from sympy import latex
from IPython.display import display

# Definir la variable
t, s = sp.symbols('t s')

# Definir la función
f1 = sp.exp(-3 * t)

# Resultado de L{e^{3t}}
L_e3t = sp.laplace_transform(f1, t, s)[0]  # Esto es 1/(s - 3)

# Imprimir la ecuación y el resultado en formato LaTeX
display(sp.Eq(sp.Function('L')(f1), L_e3t))

 ⎛ -3⋅t⎞     1  
L⎝ℯ    ⎠ = ─────
           s + 3

Actividad 4

In [59]:
# Definir alpha como una variable
alpha = sp.symbols('alpha')

# Definir la función
f2 = sp.exp(alpha * t)

# Resultado de L{e^{alpha t}}
L_ealpha_t = sp.laplace_transform(f2, t, - s)[0]  # Esto es 1/(s - alpha)

# Imprimir la ecuación y el resultado en formato LaTeX
display(sp.Eq(sp.Function('L')(f2), L_ealpha_t))

 ⎛ α⋅t⎞     1   
L⎝ℯ   ⎠ = ──────
          -α - s

Actividad 5

In [61]:
# Importar la biblioteca necesaria
import sympy as sp
from IPython.display import display

# Definir la variable
t, s = sp.symbols('t s')

# Definir la función
f = sp.sin(2 * t)

# Resultado de L{sin(2t)}
L_sin_2t = sp.laplace_transform(f, t, s)[0]  # Esto es 2 / (s^2 + 4)

# Imprimir la ecuación y el resultado en formato LaTeX
display(sp.Eq(sp.Function('L')(f, s), L_sin_2t))

                   2   
L(sin(2⋅t), s) = ──────
                  2    
                 s  + 4

Actividad 6

In [64]:
# Importar la biblioteca necesaria
import sympy as sp
from sympy import display

# Definir la variable
t, s = sp.symbols('t s')

# Definir la función
f1 = t + 5 * t

# Resultado de L{t + 5t}
L_f1 = sp.laplace_transform(f1, t, s)[0]  # Esto es 6/s^2

# Reescribir el resultado en la forma deseada
result_f1 = (1 / s) + (s / s**2)

# Imprimir la ecuación y el resultado en formato LaTeX
display(sp.Eq(sp.Function('F')(s), result_f1))

ImportError: cannot import name 'display' from 'sympy' (/usr/local/lib/python3.12/dist-packages/sympy/__init__.py)